In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import string
import re
%matplotlib inline
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is

In [39]:

amazon = pd.read_json('Prime_Pantry_5.json', lines=True)

# amazon.to_csv('Prime_Pantry_5.csv', encoding='utf-8', index=False)

# amazon = pd.read_csv('Prime_Pantry_5.csv')

amazon['reviewText'] = amazon['reviewText'].astype(str)

Roberta Analysis

In [42]:
rob_amazon = list()
# new =pd.DataFrame()
for i,  row in tqdm(amazon.iterrows(), total = len(amazon)):
    try:
#         print({i})
        text = row['reviewText']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_amazon.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text {i}')
    except IndexError:
        print(f'Index Error {i}')

  0%|          | 0/137788 [00:00<?, ?it/s]

Too Long Text 841
Index Error 975
Too Long Text 2885
Too Long Text 3799
Too Long Text 4597
Too Long Text 4984
Too Long Text 4988
Index Error 5644
Too Long Text 5761
Too Long Text 5764
Too Long Text 5778
Too Long Text 6626
Too Long Text 8779
Too Long Text 9755
Too Long Text 9922
Too Long Text 10233
Too Long Text 11054
Too Long Text 12064
Too Long Text 12232
Too Long Text 13721
Too Long Text 13778
Too Long Text 13779
Too Long Text 14004
Too Long Text 14146
Too Long Text 14154
Too Long Text 14867
Too Long Text 15908
Too Long Text 16190
Too Long Text 17369
Too Long Text 18379
Too Long Text 18547
Too Long Text 20036
Too Long Text 20093
Too Long Text 20094
Too Long Text 20319
Too Long Text 20461
Too Long Text 20469
Too Long Text 21182
Too Long Text 22223
Too Long Text 22505
Too Long Text 23261
Too Long Text 24706
Too Long Text 27366
Too Long Text 27973
Too Long Text 28081
Too Long Text 28232
Too Long Text 28652
Too Long Text 29012
Too Long Text 29522
Too Long Text 30175
Too Long Text 33302
T

In [43]:
rob_df3 = pd.DataFrame(rob_amazon)
rob_df3

,roberta_neg,roberta_neu,roberta_pos
0,0.398582,0.396397,0.205021
1,0.013206,0.099717,0.887077
2,0.013562,0.184666,0.801772
3,0.037135,0.303437,0.659429
4,0.023968,0.440897,0.535135
...,...,...,...
137629,0.058178,0.308300,0.633522
137630,0.001205,0.008453,0.990342
137631,0.549926,0.399450,0.050624
137632,0.002597,0.017653,0.979750


In [44]:
rob_amazon2 = rob_df3.join(amazon)

VADERS Analyis

In [45]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

In [46]:

# amazon = pd.read_json('Prime_Pantry_5.json', lines=True)

# amazon.to_csv('Prime_Pantry_5.csv', encoding='utf-8', index=False)

# amazon = pd.read_csv('Prime_Pantry_5.csv')

rob_amazon2['review'] = rob_amazon2['reviewText'].apply(lambda reviewText:sia.polarity_scores(str(reviewText)))

objs = [rob_amazon2, pd.DataFrame(rob_amazon2['review'].tolist()).iloc[:, :4]]
df2 = pd.concat(objs, axis=1).drop('review', axis=1)

In [47]:
# # res = {}
# v_amazon = list()
# # new =pd.DataFrame()
# for i,  row in tqdm(amazon.iterrows(), total = len(amazon)):
#     text = row['reviewText']
# #     myid = row['reviewerID']
#     res = sia.polarity_scores(text)
# #     print(res)
#     v_amazon.append(res)

In [48]:
# len(v_amazon)

In [49]:
# vaders_amazon = pd.DataFrame(v_amazon)
# vaders_amazon

Combine both datasets

In [50]:
# final_df3 = vaders_amazon.join(rob_amazon2)

In [51]:
# final_df3

In [52]:
df2.to_csv('Amazon_tweets_analysis.csv')

Cleaned

In [3]:

amazon = pd.read_json('Prime_Pantry_5.json', lines=True)

# amazon.to_csv('Prime_Pantry_5.csv', encoding='utf-8', index=False)

# amazon = pd.read_csv('Prime_Pantry_5.csv')

amazon['reviewText'] = amazon['reviewText'].astype(str)

In [4]:
# # Remove Punctuations
# def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
#     text = re.sub('[0-9]+', '', text)
#     return text

# amazon['reviewText2'] = amazon['reviewText'].apply(lambda x: remove_punct(x))

# amazon

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,reviewText2
0,4,True,"09 24, 2015",A31Y9ELLA1JUB0,B0000DIWNI,Her Royal Peepness Princess HoneyBunny Blayze,I purchased this Saran premium plastic wrap af...,Pretty Good For plastic Wrap,1443052800,NaN,NaN,NaN,I purchased this Saran premium plastic wrap af...
1,5,True,"06 23, 2015",A2FYW9VZ0AMXKY,B0000DIWNI,Mary,I am an avid cook and baker. Saran Premium Pl...,"The Best Plastic Wrap for your Cooking, Baking...",1435017600,NaN,NaN,NaN,I am an avid cook and baker Saran Premium Pla...
2,5,True,"06 13, 2015",A1NE43T0OM6NNX,B0000DIWNI,Tulay C,"Good wrap, keeping it in the fridge makes it e...",Good and strong.,1434153600,NaN,NaN,NaN,Good wrap keeping it in the fridge makes it ea...
3,4,True,"06 3, 2015",AHTCPGK2CNPKU,B0000DIWNI,OmaShops,I prefer Saran wrap over other brands. It does...,Doesn't cling as well to dishes as other brand...,1433289600,NaN,NaN,NaN,I prefer Saran wrap over other brands It doesn...
4,5,True,"04 20, 2015",A25SIBTMVXLB59,B0000DIWNI,Nitemanslim,Thanks,Five Stars,1429488000,NaN,NaN,NaN,Thanks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137783,5,True,"02 2, 2018",A2P38K4LK09134,B01HI76790,Amazon Customer,great,Five Stars,1517529600,NaN,NaN,NaN,great
137784,4,True,"11 30, 2017",AE4FU8QRB3KXA,B01HI76XS0,J,These are delicious and healthy snacks! I wit...,I purchased these because they're lower in sug...,1512000000,NaN,NaN,NaN,These are delicious and healthy snacks I with...
137785,5,True,"09 17, 2017",A36MOFABIPIPGM,B01HI76XS0,Love to Cook,Taste not to be believed. Buy a box for my off...,Five Stars,1505606400,NaN,NaN,NaN,Taste not to be believed Buy a box for my offi...
137786,5,True,"01 31, 2018",A1TKNVUVJ8I8KW,B01HI76SA8,Jacci Washington,They are yummy!,Best healthy snack ever!,1517356800,NaN,NaN,NaN,They are yummy


In [5]:
# def tokenization(text):
#     text = re.split('\W+', text)
#     return text

# amazon['reviewText3'] = amazon['reviewText2'].apply(lambda x: tokenization(x.lower()))
# amazon.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,reviewText2,reviewText3
0,4,True,"09 24, 2015",A31Y9ELLA1JUB0,B0000DIWNI,Her Royal Peepness Princess HoneyBunny Blayze,I purchased this Saran premium plastic wrap af...,Pretty Good For plastic Wrap,1443052800,NaN,NaN,NaN,I purchased this Saran premium plastic wrap af...,"[i, purchased, this, saran, premium, plastic, ..."
1,5,True,"06 23, 2015",A2FYW9VZ0AMXKY,B0000DIWNI,Mary,I am an avid cook and baker. Saran Premium Pl...,"The Best Plastic Wrap for your Cooking, Baking...",1435017600,NaN,NaN,NaN,I am an avid cook and baker Saran Premium Pla...,"[i, am, an, avid, cook, and, baker, saran, pre..."
2,5,True,"06 13, 2015",A1NE43T0OM6NNX,B0000DIWNI,Tulay C,"Good wrap, keeping it in the fridge makes it e...",Good and strong.,1434153600,NaN,NaN,NaN,Good wrap keeping it in the fridge makes it ea...,"[good, wrap, keeping, it, in, the, fridge, mak..."
3,4,True,"06 3, 2015",AHTCPGK2CNPKU,B0000DIWNI,OmaShops,I prefer Saran wrap over other brands. It does...,Doesn't cling as well to dishes as other brand...,1433289600,NaN,NaN,NaN,I prefer Saran wrap over other brands It doesn...,"[i, prefer, saran, wrap, over, other, brands, ..."
4,5,True,"04 20, 2015",A25SIBTMVXLB59,B0000DIWNI,Nitemanslim,Thanks,Five Stars,1429488000,NaN,NaN,NaN,Thanks,[thanks]


In [6]:
# stopword = nltk.corpus.stopwords.words('english')
# def remove_stopwords(text):
#     text = [word for word in text if word not in stopword]
#     return text
    
# amazon['reviewText4'] = amazon['reviewText3'].apply(lambda x: remove_stopwords(x))

In [7]:
# amazon.to_csv('amazon_pantry_tweets_cleaned.csv')

In [14]:
amazon = pd.read_csv('amazon_pantry_tweets_cleaned.csv')
amazon['reviewText4'] = amazon['reviewText4'].astype(str)

In [16]:
rob_amazon = list()
# new =pd.DataFrame()
for i,  row in tqdm(amazon.iterrows(), total = len(amazon)):
    try:
#         print({i})
        text = row['reviewText4']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_amazon.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text {i}')
    except IndexError:
        print(f'Index Error {i}')

  0%|          | 0/137788 [00:00<?, ?it/s]

Too Long Text 9755
Too Long Text 12064
Too Long Text 18379
Too Long Text 33393
Too Long Text 33950
Too Long Text 34045
Too Long Text 38834
Too Long Text 41690
Too Long Text 41930
Too Long Text 42120
Too Long Text 101198
Too Long Text 105946


In [17]:
rob_df3 = pd.DataFrame(rob_amazon)

In [18]:
rob_amazon2 = rob_df3.join(amazon)

In [19]:
rob_amazon2['reviewText5'] = rob_amazon2['reviewText4'].apply(lambda reviewText4:sia.polarity_scores(str(reviewText4)))

objs = [rob_amazon2, pd.DataFrame(rob_amazon2['reviewText5'].tolist()).iloc[:, :4]]
rob_amazon3 = pd.concat(objs, axis=1).drop('reviewText5', axis=1)

In [20]:
rob_amazon3.to_csv('amazon_pantry_tweets_analysis_cleaned.csv')